# Experiment Tracking

## Introduction
- Ml Experiment: the process of building an ML model
- Experiment run: each trial in an ML experiment
- Run artifact: any file that is associated with an ML run
- Experiment metadata: all info asscociated with the model

Experiment tracking is the process of keeping track of all the relevant info from an ML experiment, which includes:
- Source Code
- Environment
- Data
- Model
- Hyperparameters
- Metric

It is so important because:
- Reproducibility
- Organization
- Optimization


## MLflow
An open source platform for the ML lifecycle. It is basically a python package that contains 4 modules:
- Tracking: this module helps with the organization of experiments into runs, and it keeps track of:
    - parameters: includes hyperparameter, path to the data, etc. it includes anything that affects the metrics of the model
    - metrics: such as accuracy, metrics from the test or validation
    - metadata: via tags such as developer names
    - artifacts: such as visualization, etc
    - models

        MLflow automatically logs extra information about the run such as:
        - Source code
        - Version of the code (git commit)
        - Start and end time
        - Author

- Models
- Model Registry
- Projects


